# Задание
Рассчитать данные по энергопотреблению
Инструкция
Загрузите данные и посчитайте модели линейной регрессии для 50 зданий по ансамблю регрессионных моделей: в первой модели весь оптимальный набор метеорологических данных, во второй - дни недели и праздники, в третьей - недели года, в четвертой - месяцы. Финальное значение показателя рассчитайте как взвешенное арифметическое среднее показателей всех моделей, взяв веса для первой и второй модели как 3/8, а для третьей и четвертой - как 1/8.

Загрузите данные решения, посчитайте значение энергопотребления для требуемых дат для тех зданий, которые посчитаны в модели, и выгрузите результат в виде CSV-файла (submission.csv).

Данные:
* http://video.ittensive.com/machine-learning/ashrae/building_metadata.csv.gz
* http://video.ittensive.com/machine-learning/ashrae/weather_train.csv.gz
* http://video.ittensive.com/machine-learning/ashrae/train.0.csv.gz
* http://video.ittensive.com/machine-learning/ashrae/test.csv.gz
* http://video.ittensive.com/machine-learning/ashrae/weather_test.csv.gz

Итоговый файл с кодом (.py или .ipynb) выложите в github с портфолио.

In [1]:
# Подключение библиотек
from scipy.interpolate import interp1d
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from pandas.tseries.holiday import USFederalHolidayCalendar as calendar
import os
from pandas.io import pytables
import gzip

In [2]:
# Загрузка данных, отсечение 20 зданий, объединение и оптимизация

def reduce_memory_usage (df):
    
    start_mem = df.memory_usage().sum() / 1024**2
    for col in df.columns:
        col_type = df[col].dtypes
        if str(col_type)[:5] == "float":
            c_min = df[col].min()
            c_max = df[col].max()
            if c_min > np.finfo("f2").min and c_max < np.finfo("f2").max:
                df[col] = df[col].astype(np.float16)
            elif c_min > np.finfo("f4").min and c_max < np.finfo("f4").max:
                df[col] = df[col].astype(np.float32)
            else:
                df[col] = df[col].astype(np.float64)
        elif str(col_type)[:3] == "int":
            c_min = df[col].min()
            c_max = df[col].max()
            if c_min > np.iinfo("i1").min and c_max < np.iinfo("i1").max:
                df[col] = df[col].astype(np.int8)
            elif c_min > np.iinfo("i2").min and c_max < np.iinfo("i2").max:
                df[col] = df[col].astype(np.int16)
            elif c_min > np.iinfo("i4").min and c_max < np.iinfo("i4").max:
                df[col] = df[col].astype(np.int32)
            else:
                df[col] = df[col].astype(np.int64)
        elif col == "timestamp":
            df[col] = pd.to_datetime(df[col])
        elif str(col_type)[:8] != "datetime":
            df[col] = df[col].astype("category")
    end_mem = df.memory_usage().sum() / 1024**2
    print ("Потребление памяти меньше на",
          round(start_mem - end_mem, 2),
          "Мб (минус",
           round(100 * (start_mem - end_mem) / start_mem, 1),
           "%)")
    return df

In [3]:
buildings = pd.read_csv("http://video.ittensive.com/machine-learning/ashrae/building_metadata.csv.gz")
weather = pd.read_csv("http://video.ittensive.com/machine-learning/ashrae/weather_train.csv.gz")
weather = weather[weather["site_id"] == 0]
energy = pd.read_csv("http://video.ittensive.com/machine-learning/ashrae/train.0.csv.gz",compression='gzip', encoding ='utf-8')
energy = energy[energy["building_id"] < 50]
energy = pd.merge(left=energy, right=buildings, how="left",
                 left_on="building_id", right_on="building_id", suffixes=('', '_remove'))
del buildings
print (energy.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 432456 entries, 0 to 432455
Data columns (total 9 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   building_id    432456 non-null  int64  
 1   meter          432456 non-null  int64  
 2   timestamp      432456 non-null  object 
 3   meter_reading  432456 non-null  float64
 4   site_id        432456 non-null  int64  
 5   primary_use    432456 non-null  object 
 6   square_feet    432456 non-null  int64  
 7   year_built     432456 non-null  float64
 8   floor_count    0 non-null       float64
dtypes: float64(3), int64(4), object(2)
memory usage: 33.0+ MB
None


In [4]:
# Интерполяция значений
weather["precip_depth_1_hr"] = weather["precip_depth_1_hr"].apply(lambda x:x if x>0 else 0)
interpolate_columns = ["air_temperature", "dew_temperature", 
                      "cloud_coverage", "wind_speed", "precip_depth_1_hr", 
                      "sea_level_pressure"]
for col in interpolate_columns:
    weather[col] = weather[col].interpolate(limit_diraction='both',
                                         kind='cubic')

In [5]:
# Обогащение данных: погода

weather["air_temperature_diff1"] = weather["air_temperature"].diff()
weather.at[0,"air_temperature_diff1"] = weather.at[1,"air_temperature_diff1"]
weather["air_temperature_diff2"] = weather["air_temperature_diff1"].diff()
weather.at[0,"air_temperature_diff2"] = weather.at[1,"air_temperature_diff2"]

In [6]:
# Объединение погодных данных
energy = energy.set_index(["timestamp", "site_id"])
weather = weather.set_index(["timestamp", "site_id"])
energy = pd.merge(left=energy, right=weather, how="left",
                 left_index=True, right_index=True)
energy.reset_index(inplace=True)
energy = energy.drop(columns=["site_id"], axis=1)
energy = reduce_memory_usage(energy)
del weather
print (energy.info())

Потребление памяти меньше на 37.53 Мб (минус 66.9 %)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 432456 entries, 0 to 432455
Data columns (total 17 columns):
 #   Column                 Non-Null Count   Dtype         
---  ------                 --------------   -----         
 0   timestamp              432456 non-null  datetime64[ns]
 1   building_id            432456 non-null  int8          
 2   meter                  432456 non-null  int8          
 3   meter_reading          432456 non-null  float16       
 4   primary_use            432456 non-null  category      
 5   square_feet            432456 non-null  int32         
 6   year_built             432456 non-null  float16       
 7   floor_count            0 non-null       float64       
 8   air_temperature        432456 non-null  float16       
 9   cloud_coverage         432456 non-null  float16       
 10  dew_temperature        432456 non-null  float16       
 11  precip_depth_1_hr      432456 non-null  float16    

In [7]:
# Обогащение данных: дата
# для них добавим дни недели,недели и праздничные дни

energy["hour"] = energy["timestamp"].dt.hour.astype("int8")
energy["weekday"] = energy["timestamp"].dt.weekday.astype("int8")
energy["week"] = energy["timestamp"].dt.isocalendar().week.astype("int8")
energy["month"] = energy["timestamp"].dt.month.astype("int8")
energy["date"] = pd.to_datetime(energy["timestamp"].dt.date)
dates_range = pd.date_range(start='2015-12-31', end='2017-01-01')
us_holidays = calendar().holidays(start=dates_range.min(),
                                 end=dates_range.max())
energy["is_holiday"] = energy["date"].isin(us_holidays).astype("int8")
# для каждой строки данных введем параметры-флаги по принадлежности данных к определенному дню недели 
# и определённой неделе в году. Это должно помочь скорректировать предсказание энергопотребления по
# сезонам и в будни и в выходные
for weekday in range(0, 7):
    energy["is_wday" + str(weekday)] = energy["weekday"].isin([weekday]).astype("int8")
for week in range(1, 54):
    energy["is_w" + str(week)] = energy["week"].isin([week]).astype("int8")
for month in range(1, 13):
    energy["is_m" + str(month)] = energy["month"].isin([month]).astype("int8")

In [8]:
# Логарифмирование данных
# дополнительно проведем логарифмирование ключевого показателя для проверки, можно ли перейти от линейной
# модели суммы показателей к модели призведения
# z= A*x + B*y logz =A*x + B*y z=e^Ax*e^By z=a^x*b^y
energy["meter_reading_log"] = np.log(energy["meter_reading"] + 1)

In [9]:
# Экспорт данных в CSV и HDF5

energy.to_csv("energy.0-50.ready.csv.gz", index=False)
print (energy.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 432456 entries, 0 to 432455
Data columns (total 96 columns):
 #   Column                 Non-Null Count   Dtype         
---  ------                 --------------   -----         
 0   timestamp              432456 non-null  datetime64[ns]
 1   building_id            432456 non-null  int8          
 2   meter                  432456 non-null  int8          
 3   meter_reading          432456 non-null  float16       
 4   primary_use            432456 non-null  category      
 5   square_feet            432456 non-null  int32         
 6   year_built             432456 non-null  float16       
 7   floor_count            0 non-null       float64       
 8   air_temperature        432456 non-null  float16       
 9   cloud_coverage         432456 non-null  float16       
 10  dew_temperature        432456 non-null  float16       
 11  precip_depth_1_hr      432456 non-null  float16       
 12  sea_level_pressure     432456 non-null  floa

In [10]:
buildings = pd.read_csv("http://video.ittensive.com/machine-learning/ashrae/building_metadata.csv.gz",
                       usecols=["site_id", "building_id"])
weather = pd.read_csv("http://video.ittensive.com/machine-learning/ashrae/weather_test.csv.gz")
print (weather.info())
weather = weather[weather["site_id"] == 0]
results = pd.read_csv("http://video.ittensive.com/machine-learning/ashrae/test.csv.gz")
#print (results.info())
results = results[(results["building_id"] < 50) & (results["meter"] == 0)] 
results = pd.merge(left=results, right=buildings, how="left",
                  left_on="building_id", right_on="building_id")

del buildings
results = results.drop(columns=["meter"], axis=1)
print (results.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 277243 entries, 0 to 277242
Data columns (total 9 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   site_id             277243 non-null  int64  
 1   timestamp           277243 non-null  object 
 2   air_temperature     277139 non-null  float64
 3   cloud_coverage      136795 non-null  float64
 4   dew_temperature     276916 non-null  float64
 5   precip_depth_1_hr   181655 non-null  float64
 6   sea_level_pressure  255978 non-null  float64
 7   wind_direction      264873 non-null  float64
 8   wind_speed          276783 non-null  float64
dtypes: float64(7), int64(1), object(1)
memory usage: 19.0+ MB
None
<class 'pandas.core.frame.DataFrame'>
Int64Index: 876000 entries, 0 to 875999
Data columns (total 4 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   row_id       876000 non-null  int64 
 1   building_id  876000 non-null  i

In [11]:
# Интерполяция значений и обогащение погодных данных: только для одного города

interpolate_columns = ["air_temperature", "dew_temperature", 
                      "cloud_coverage", "wind_speed",
                      "sea_level_pressure"]
for col in interpolate_columns:
    weather[col] = weather[col].interpolate(limit_direction='both',
                                            kind ='cubic')
weather["air_temperature_diff1"] = weather["air_temperature"].diff()                                           
weather.at[0, "air_temperature_diff1"] = weather.at[1, "air_temperature_diff1"]
weather["air_temperature_diff2"] = weather["air_temperature_diff1"].diff()                                           
weather.at[0, "air_temperature_diff2"] = weather.at[1, "air_temperature_diff2"]

In [12]:
# Объединение данных по погоде

results = results.set_index(["timestamp", "site_id"])
weather = weather.set_index(["timestamp", "site_id"])
results = pd.merge(left=results, right=weather, how='left',
                  left_index=True, right_index=True)
results.reset_index(inplace=True)
results = results.drop(columns=["site_id"], axis=1)

del weather
results = reduce_memory_usage(results)
print (results.info())

Потребление памяти меньше на 54.3 Мб (минус 67.7 %)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 876000 entries, 0 to 875999
Data columns (total 12 columns):
 #   Column                 Non-Null Count   Dtype         
---  ------                 --------------   -----         
 0   timestamp              876000 non-null  datetime64[ns]
 1   row_id                 876000 non-null  int32         
 2   building_id            876000 non-null  int8          
 3   air_temperature        876000 non-null  float16       
 4   cloud_coverage         876000 non-null  float16       
 5   dew_temperature        876000 non-null  float16       
 6   precip_depth_1_hr      874500 non-null  float16       
 7   sea_level_pressure     876000 non-null  float16       
 8   wind_direction         852150 non-null  float16       
 9   wind_speed             876000 non-null  float16       
 10  air_temperature_diff1  876000 non-null  float16       
 11  air_temperature_diff2  876000 non-null  float16     

In [13]:
# Обогащение данных по дате

results["hour"] = results["timestamp"].dt.hour.astype("int8")
results["weekday"] = results["timestamp"].dt.weekday.astype("int8")
results["week"] = results["timestamp"].dt.isocalendar().week.astype("int8")
results["month"] = results["timestamp"].dt.month.astype("int8")
results["date"] = pd.to_datetime(results["timestamp"].dt.date)
dates_range = pd.date_range(start='2015-12-31', end='2017-01-01')
us_holidays = calendar().holidays(start=dates_range.min(),
                                 end=dates_range.max())
results["is_holiday"] = results["date"].isin(us_holidays).astype("int8")
for weekday in range(0, 7):
    results["is_wday" + str(weekday)] = results["weekday"].isin([weekday]).astype("int8")
for week in range(1, 54):
    results["is_w" + str(week)] = results["week"].isin([week]).astype("int8")
for month in range(1, 13):
    results["is_m" + str(month)] = results["month"].isin([month]).astype("int8")

In [14]:
# Обозначим набор параметров для каждой модели

lr_weather_columns = ["meter_reading_log", "hour", "building_id",
                     "air_temperature", "dew_temperature",
                     "sea_level_pressure", "wind_speed",
                     "air_temperature_diff1", "air_temperature_diff2",
                     "cloud_coverage"]
lr_days_columns = ["meter_reading_log", "hour", "building_id",
                  "is_holiday"]
lr_week_columns = ["meter_reading_log", "hour", "building_id",
                   "week"]
lr_month_columns = ["meter_reading_log", "hour", "building_id",
                   "month"]
for wday in range(0, 7):
    lr_days_columns.append("is_wday" + str(wday))
    print (lr_days_columns)
for week in range(1, 54):
    lr_week_columns.append("is_w" + str(week))
    print (lr_week_columns)
for month in range(1, 13):
    lr_month_columns.append("is_m" + str(month))
    print (lr_month_columns)
hours = range(0, 24)
buildings = range(0, energy["building_id"].max() + 1)

['meter_reading_log', 'hour', 'building_id', 'is_holiday', 'is_wday0']
['meter_reading_log', 'hour', 'building_id', 'is_holiday', 'is_wday0', 'is_wday1']
['meter_reading_log', 'hour', 'building_id', 'is_holiday', 'is_wday0', 'is_wday1', 'is_wday2']
['meter_reading_log', 'hour', 'building_id', 'is_holiday', 'is_wday0', 'is_wday1', 'is_wday2', 'is_wday3']
['meter_reading_log', 'hour', 'building_id', 'is_holiday', 'is_wday0', 'is_wday1', 'is_wday2', 'is_wday3', 'is_wday4']
['meter_reading_log', 'hour', 'building_id', 'is_holiday', 'is_wday0', 'is_wday1', 'is_wday2', 'is_wday3', 'is_wday4', 'is_wday5']
['meter_reading_log', 'hour', 'building_id', 'is_holiday', 'is_wday0', 'is_wday1', 'is_wday2', 'is_wday3', 'is_wday4', 'is_wday5', 'is_wday6']
['meter_reading_log', 'hour', 'building_id', 'week', 'is_w1']
['meter_reading_log', 'hour', 'building_id', 'week', 'is_w1', 'is_w2']
['meter_reading_log', 'hour', 'building_id', 'week', 'is_w1', 'is_w2', 'is_w3']
['meter_reading_log', 'hour', 'buildin

In [15]:
# Введём ф-ию для вычисления качества моделей
import warnings
warnings.filterwarnings('ignore')
def calculate_model (x, df_lr, lr_columns):
    lr = -1
    model = df_lr[x.building_id][x.hour]
    if len(model) > 0:
        lr = np.sum([x[col]*model[i] for i, col in enumerate(lr_columns[3:])])
        lr += model[len(lr_columns)-3]
        
        lr = np.exp(lr)
    if lr < 0 or lr*lr == lr:
        lr = 0
    x["meter_reading_lr_q"] = (np.log(x.meter_reading + 1) + 
                              np.log(1 + lr))**2    
    return x

In [16]:
# Введем ф-ии для разделения данных, построение моделей и вычисления
# их кач-ва(для обновления весов ансамбля)
# Ансамбль моделей линейной регрессии: Z=A*погода + B*дни недели,A+B=1 

def train_model (df, columns):
    df_train_lr = pd.DataFrame(df, columns=columns)
    df_lr = [[]]*len(buildings)
    for building in buildings:
        df_lr[building] = [[]]*len(hours)
        df_train_b = df_train_lr[df_train_lr["building_id"] == building]
        for hour in hours:
            df_train_bh = df_train_b[df_train_b["hour"] == hour]
            y = df_train_bh["meter_reading_log"]
            x = df_train_bh.drop(labels=["meter_reading_log", 
                    "hour", "building_id"], axis=1)
            x = x.replace((np.inf, -np.inf, np.nan), 0).reset_index(drop=True)
            model = LinearRegression(fit_intercept=False).fit(x, y)
            df_lr[building][hour] = model.coef_
            df_lr[building][hour] = np.append(df_lr[building][hour],
                                             model.intercept_)
    return df_lr

def calculate_weights_model(df_test, df_train, lr_columns):
    df_test = df_test.apply(calculate_model, axis=1, 
                result_type="expand",
                df_lr=train_model(df_train, lr_columns),
                lr_columns=lr_columns)
    return pd.Series(df_test.groupby(["hour",
            "building_id"]).sum(numeric_only=True)["meter_reading_lr_q"])

def calculate_weights():
    df_train, df_test = train_test_split(energy[energy["meter_reading"] > 0],
                                        test_size=0.2)
    return (calculate_weights_model(df_test, df_train, lr_weather_columns),
            calculate_weights_model(df_test, df_train, lr_days_columns),
            calculate_weights_model(df_test, df_train, lr_week_columns),
            calculate_weights_model(df_test, df_train, lr_month_columns))

In [17]:
# Рассчитаем оптимальные веса для каждого часа и здания
 
#рассчитаем в каждом случае значения ошибки для каждого здания и часа.
# Сформируем список весов: 1 - учитываем регрессию по дням, 0-по погоде
# рассмотрим два варианта ансамбля моделей линейной регрессии: 
# либо среднее арифметическое всех моделей, либо выбор лучшей модели из двух для конкретного здания и часа

weights_weather = []
weights_wday = []
weights_week = []
weights_month = []

weights_weather_model, weights_wday_model, weights_week_model, weights_month_model = calculate_weights()
if len(weights_weather) > 0:
    weights_weather = weights_weather + weights_weather_model
else:
    weights_weather = weights_weather_model
if len(weights_wday) > 0:
    weights_wday = weights_wday + weights_wday_model
else:
    weights_wday = weights_wday_model 
if len(weights_week) > 0:
    weights_week = weights_week + weights_week_model
else:
    weights_week = weights_week_model
if len(weights_month) > 0:
    weights_month = weights_month + weights_month_model
else:
    weights_month = weights_month_model      

weight_weather = weights_weather[0:19]
weight_wday = weights_wday[0:19]
weights_week = weights_week[0:7]
weights_month = weights_month[0:7]

In [18]:
# Посчитаем ансамбль линейной регрессии
# разделим данные на обучающие/тестовые

energy_train, energy_test = train_test_split(energy[energy["meter_reading"] > 0],
                                            test_size=0.2)

In [19]:
# Обучим модели линейной регрессии по погоде,дням недели,неделям,месяцам

energy_lr_days = train_model(energy_train, lr_days_columns)
energy_lr_weather = train_model(energy_train, lr_weather_columns)
energy_lr_week = train_model(energy_train, lr_week_columns)
energy_lr_month = train_model(energy_train, lr_month_columns)

In [20]:
# Рассчитаем финальное качество ансамбля


def calculate_model_ensemble (x, model, columns):
    lr = -1
    if len(model) > 0:
        lr = np.sum([x[col]*model[i] for i, col in enumerate(columns[3:])])
        lr += model[len(columns)-3]
        lr = np.exp(lr)
    if lr < 0 or lr*lr == lr:
        lr = 0
    return lr

def calculate_models_ensemble (x):
    lr_d = calculate_model_ensemble(x,
                energy_lr_days[x.building_id][x.hour],
                lr_days_columns)
    lr_weather = calculate_model_ensemble(x,
                energy_lr_weather[x.building_id][x.hour],
                lr_weather_columns)
    lr_w = calculate_model_ensemble(x,
                energy_lr_week[x.building_id][x.hour],
                lr_week_columns)
    lr_m = calculate_model_ensemble(x,
                energy_lr_month[x.building_id][x.hour],
                lr_month_columns)
    lr_sum = (lr_d + lr_weather + lr_w + lr_m)/4
    
    x["meter_reading_sum_q"] = (np.log(x.meter_reading +1) +
                               np.log(1+lr_sum))**2
    return x
energy_test = energy_test.apply(calculate_models_ensemble,
                               axis=1, result_type="expand")

In [21]:
# Усечение данных до требуемого формата:row_id,meter_reading


energy_test_ready = pd.DataFrame(energy_test, columns=["row_id", "meter_reading"])

In [22]:
# Загрузка всех данных для заполнения их нулями

energy_test = pd.read_csv("http://video.ittensive.com/machine-learning/ashrae/test.csv.gz",
                     usecols=["row_id"])
energy_test = pd.merge(left=energy_test, right=energy_test_ready, how='left',
                  left_on="row_id", right_on="row_id")
energy_test.fillna(value=0, inplace=True)
print (results.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 876000 entries, 0 to 875999
Data columns (total 90 columns):
 #   Column                 Non-Null Count   Dtype         
---  ------                 --------------   -----         
 0   timestamp              876000 non-null  datetime64[ns]
 1   row_id                 876000 non-null  int32         
 2   building_id            876000 non-null  int8          
 3   air_temperature        876000 non-null  float16       
 4   cloud_coverage         876000 non-null  float16       
 5   dew_temperature        876000 non-null  float16       
 6   precip_depth_1_hr      874500 non-null  float16       
 7   sea_level_pressure     876000 non-null  float16       
 8   wind_direction         852150 non-null  float16       
 9   wind_speed             876000 non-null  float16       
 10  air_temperature_diff1  876000 non-null  float16       
 11  air_temperature_diff2  876000 non-null  float16       
 12  hour                   876000 non-null  int8

In [23]:
# Выгрузка результатов в CSV файл

energy_test.to_csv("submission.csv")

In [24]:
# Освобождение памяти
del energy
del energy_test
del energy_test_ready